In [1]:
import pandas as pd
import datetime
import requests
import Module_date
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

    
def generate_calendar_excel(process_date, make_up_day, not_holiday_list, special_holiday_list):
    # 獲取節假日數據
    holiday_df = Module_date.get_holiday_df(process_date)
    holiday_df['Date'] = pd.to_datetime(holiday_df['date']).dt.strftime('%Y-%m-%d')
    holiday_df = holiday_df[holiday_df['Date'] >= process_date.strftime('%Y-%m-01')]
    # 處理節假日相關信息
    holiday_df = Module_date.process_holidays(holiday_df, make_up_day, not_holiday_list, special_holiday_list)
    
    # 創建日期範圍 DataFrame
    date_range = pd.date_range(start=process_date, end=f"{process_date.year}-12-31")
    date_df = pd.DataFrame({'Date': date_range})
    date_df['Date'] = date_df['Date'].dt.strftime('%Y-%m-%d')
    
    # 合併節假日數據和日期範圍
    res_df = date_df.merge(holiday_df, 'left', 'Date').fillna('')
    res_df['Date'] = pd.to_datetime(res_df['Date'])
    res_df['Category'] = res_df.apply(lambda row: row['holidaycategory'] if row['isholiday'] == '是' else '上班日', axis=1)
    res_df['BreakTime'] = '01:00'
    res_df['ArrivalTime'] = res_df['isholiday'].apply(lambda x: '假日' if x == '是' else '')
    res_df['DepartureTime'] = res_df['isholiday'].apply(lambda x: '假日' if x == '是' else '')
    res_df['WorkHours'] = ''
    res_df['HoursGap'] = ''
    res_df['MinutesGap'] = ''
    columns_to_export = ['Date', 'Category', 'ArrivalTime', 'DepartureTime', 'BreakTime', 'WorkHours', 'HoursGap', 'MinutesGap']
    res_df = res_df[columns_to_export]
    
    # 創建 Excel 文件
    output_file = f'./result/calendar_{process_date.year}.xlsx'
    writer = pd.ExcelWriter(output_file, engine='xlsxwriter')
    workbook = writer.book
    
    # 定義 Excel 格式
    header_format = workbook.add_format({
        'bold': True,
        'text_wrap': False,
        'align': 'center',
        'border': 2,
        'font_name': 'Segoe UI',
        'font_size': 12
    })
    
    cell_format = workbook.add_format({
        'bold': False,
        'text_wrap': False,
        'align': 'center',
        'border': 1,
        'font_name': 'Segoe UI',
        'font_size': 12
    })
    
    Date_format = workbook.add_format({
        'bold': False,
        'text_wrap': False,
        'align': 'center',
        'border': 1,
        'font_name': 'Segoe UI',
        'font_size': 12,
        'num_format': 'yyyy-m-d'
    })
    
    # 將數據寫入 Excel
    for name, group in res_df.groupby(pd.to_datetime(res_df['Date']).dt.strftime('%Y%m')):
        sheet_name = name
        group.to_excel(writer, sheet_name=sheet_name, index=False)
        (max_row, max_col) = group.shape

        worksheet = writer.sheets[sheet_name]
        worksheet.write_row(0, 0, group.columns, header_format)
        for row in range(0, max_row):
            for col in range(0, max_col):
                if col == 0:
                    worksheet.write(row + 1, col, group.iloc[row, col], Date_format)
                elif col == 5:
                    worksheet.write(row+1, col, 
                    f'''=
                    IF(AND(B{row + 2}="上班日", ISTEXT(C{row + 2})=TRUE),  TIMEVALUE("8:00"),
                        IF(B{row + 2}="加班",(D{row + 2}-C{row + 2}),
                        IF(OR(B{row + 2}<>"上班日", AND(B{row + 2}="上班日", C{row + 2}="")), "",
                        (D{row + 2}-E{row + 2}-C{row + 2}))))''', cell_format)
                    
                elif col == 6:
                    worksheet.write(row + 1, col, 
                    f'=IF(F{row + 2}="","",F{row + 2}*24)', cell_format)
                elif col == 7:
                    worksheet.write(row + 1, col, 
                    f'''=
                    IF(AND(G{row + 2}<>"",G{row + 2}="上班日"),(G{row + 2}-8)*60,
                        IF(AND(G{row + 2}<>"",B{row + 2}="加班"),G{row + 2}*60,
                    ""))''', cell_format)
                else:
                    worksheet.write(row + 1, col, group.iloc[row, col], cell_format)
        worksheet.merge_range(f'A{max_row + 2}:E{max_row + 2}',
                              f'''=
            CONCATENATE("需要總時數：", (COUNTIF(B2:B{max_row + 1},"上班日"))*8, " 小時") 
            & "，須補時數：" & ROUND(COUNTIF(B2:B{max_row + 1},"上班日")*8-G{max_row + 2}, 3)
            ''', header_format)
        worksheet.write(f'F{max_row + 2}', f'=COUNT(F2:F{max_row + 1})*8', header_format)
        worksheet.write(f'G{max_row + 2}', f'=SUM(G2:G{max_row + 1})', header_format)
        worksheet.write(f'H{max_row + 2}', f'=SUM(H2:H{max_row + 1})', header_format)
        worksheet.set_column(0, max_col + 1, 18)
    
    writer.close()
    print(f"Excel 文件 {output_file} 已成功建立")


In [3]:
process_date = datetime.date(2026,1,1)
make_up_day = False # 需要補班=True, 不需要補班=False
not_holiday_list = []
special_holiday_list = ['2026-01-02']
# 使用 generate_calendar_excel 函數生成 Excel 文件
generate_calendar_excel(process_date, make_up_day, not_holiday_list, special_holiday_list)

Excel 文件 ./result/calendar_2026.xlsx 已成功建立
